# MNIST 데이터로 MultiLayer Perceptron 구현

## TensorFlow

참고자료
- 머신러닝 초보를 위한 MNIST (https://codeonweb.com/entry/12045839-0aa9-4bad-8c7e-336b89401e10)
- TensorFlow 튜토리얼 (https://github.com/golbin/TensorFlow-Tutorials/blob/master/06%20-%20MNIST/01%20-%20MNIST.py)

In [9]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
# 텐서플로우에 기본 내장된 mnist 모듈을 이용하여 데이터를 로드함
# 지정한 폴더에 MNIST 데이터가 없는 경우 자동으로 데이터를 다운로드
# one_hot 옵션은 one-hot encoding을 수행함
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [10]:
print("훈련 이미지 :",  mnist.train.images.shape)
print("훈련 라벨:",  mnist.train.labels.shape)
print("테스트 이미지 : ", mnist.test.images.shape)
print("테스트 라벨 : ", mnist.test.labels.shape)
print("검증 이미지 : ", mnist.validation.images.shape)
print("검증 라벨 : ", mnist.validation.labels.shape)

훈련 이미지 : (55000, 784)
훈련 라벨: (55000, 10)
테스트 이미지 :  (10000, 784)
테스트 라벨 :  (10000, 10)
검증 이미지 :  (5000, 784)
검증 라벨 :  (5000, 10)


신경망 모델 구성

In [11]:
# 입력 값의 차원: [배치크기, 특성값]
# 손글씨 이미지는 28x28 픽셀로 이루어져 있고, 이를 784개의 특성값으로 정함
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10]) # 결과는 0~9 의 10 가지 분류를 가짐

In [12]:
# 신경망의 레이어
# 784(입력 특성값)
#   -> 256 (히든레이어 뉴런 갯수) -> 256 (히든레이어 뉴런 갯수)
#   -> 10 (결과값 0~9 분류)

W1 = tf.Variable(tf.random_normal([784, 256], stddev=0.01))
L1 = tf.nn.relu(tf.matmul(X, W1)) # 입력값에 가중치를 곱하고 ReLU 함수를 이용하여 레이어를 만듦

In [13]:
W2 = tf.Variable(tf.random_normal([256, 256], stddev=0.01))
L2 = tf.nn.relu(tf.matmul(L1, W2)) # L1 레이어의 출력값에 가중치를 곱하고 ReLU 함수를 이용하여 레이어를 만듭니다.

In [14]:
W3 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L2, W3) # 최종 모델의 출력값은 W3 변수를 곱해 10개의 분류를 가지게 됩니다.

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

신경망 모델 학습

In [15]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [16]:
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(15):
    total_cost = 0

    for i in range(total_batch):
        # 텐서플로우의 mnist 모델의 next_batch 함수를 이용해
        # 지정한 크기만큼 학습할 데이터를 가져옵니다.
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)

        _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs, Y: batch_ys})
        total_cost += cost_val

    print('Epoch:', '%04d' % (epoch + 1),
          'Avg. cost =', '{:.3f}'.format(total_cost / total_batch))

Epoch: 0001 Avg. cost = 0.410
Epoch: 0002 Avg. cost = 0.155
Epoch: 0003 Avg. cost = 0.099
Epoch: 0004 Avg. cost = 0.071
Epoch: 0005 Avg. cost = 0.052
Epoch: 0006 Avg. cost = 0.042
Epoch: 0007 Avg. cost = 0.032
Epoch: 0008 Avg. cost = 0.026
Epoch: 0009 Avg. cost = 0.019
Epoch: 0010 Avg. cost = 0.020
Epoch: 0011 Avg. cost = 0.016
Epoch: 0012 Avg. cost = 0.015
Epoch: 0013 Avg. cost = 0.013
Epoch: 0014 Avg. cost = 0.011
Epoch: 0015 Avg. cost = 0.007


결과 확인

In [17]:
# model 로 예측한 값과 실제 레이블인 Y의 값을 비교
# tf.argmax 함수를 이용해 예측한 값에서 가장 큰 값을 예측한 레이블이라고 평가
# 예) [0.1 0 0 0.7 0 0.2 0 0 0 0] -> 3
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도:', sess.run(accuracy,
                        feed_dict={X: mnist.test.images,
                                   Y: mnist.test.labels}))

정확도: 0.9779
